# Applied Data Science Capstone
## Written by Abiola David Obembe
### Created : 02-11- 2019
### Last Update : 20-02-2020
####  This notebook will be used for the capstone project (Week 3 Assignment)

In [1]:

#import required libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
import json
import requests
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
print("Hello Capstone Project Course!")
print('Libraries imported!')

Hello Capstone Project Course!
Libraries imported!


# Question 1 starts here

In [2]:
# Question 1: lunch url, scrape web page and create raw dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page= requests.get(url)
raw_df = pd.read_html(url)[0]
print(type(raw_df))
raw_df.head()

<class 'pandas.core.frame.DataFrame'>


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# Question 1:  Clean and wrangle dataframe
# drop rowa with borough not assigned
raw_df= raw_df[raw_df['Borough']!= 'Not assigned'].reset_index(drop=True)
# merge postcodes with neighbourhood
clean_df = raw_df.groupby(['Postcode','Borough']).agg({'Neighbourhood' : ', '.join}).reset_index()
# Replace not assigned entries in neigourhood column with entry in Borough
clean_df.loc[(clean_df.Neighbourhood == 'Not assigned'), 'Neighbourhood'] = clean_df['Borough']
print( 'The number of rows in the dataframe is:', clean_df.shape[0])

The number of rows in the dataframe is: 103


In [4]:
# Examine created dataframe
clean_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Question 2 starts here

In [5]:
# Question 2: download geospatial cordiantes 
!wget -q -O 'toronto_geo.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [6]:
# Question 2: read csv file to a dataframe
tor_geo_df = pd.read_csv('toronto_geo.csv')
# Check number of entries in dataset
print('Number of rows in dataset is:', tor_geo_df.shape[0])
# Examine dataframe
tor_geo_df.tail()

Number of rows in dataset is: 103


,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [7]:
# Question 2 : Merge both dataframe to give main dataframe for analysis
tor_merged = clean_df
tor_merged  = tor_merged.join(tor_geo_df.set_index('Postal Code'), on = 'Postcode')
tor_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Start of Question 3

In [8]:
# Question 3: select all rows in Bourough that contain data that match the string toronto
tor_merged_df = tor_merged[tor_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_merged_df.shape

(39, 5)

In [9]:
# We focus on Toronto location and obtain latitude and longitude using geocoder
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = "tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograhpical cordinate of Toronto are {},{}.'. format (latitude, longitude))

The geograhpical cordinate of Toronto are 43.653963,-79.387207.


In [10]:
#Question 3: Visualize boroughs that contain the word toronto using folium visualization
map_toronto = folium.Map(location =[latitude,longitude], zoom_start = 10)
# add markers to map
for lat,lng, borough, neighbourhood in zip (tor_merged_df['Latitude'], tor_merged_df['Longitude'], tor_merged_df['Borough'], tor_merged_df['Neighbourhood']):
    label = '{},{}'. format(neighbourhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, color = 'blue', fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False). add_to(map_toronto)

map_toronto

# Question 3: Using Foursquare API to explore the neighbourhoods with the word 'toronto'
## input fourquare credentials and version

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
# Question 3(contd.) : Let's explore the second neighourhood in the tor_merged_df 

neighbourhood_name = tor_merged_df.loc[1, 'Neighbourhood']
neighbourhood_latitude = tor_merged_df.loc[1, 'Latitude']
neighbourhood_longitude = tor_merged_df.loc[1,'Longitude']

print('Latitude and longitude values of {} are {},{}.'. format (neighbourhood_name, neighbourhood_latitude, neighbourhood_longitude))


Latitude and longitude values of The Danforth West, Riverdale are 43.6795571,-79.352188.


In [13]:
# Question 3(contd.) : define function to get venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
# Question 3(contd.) : Let's obtain the top 50 venues in the above location within a radius of 500 meters

LIMIT = 50
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v= {}&ll={},{}&radius={}&limit={}'.format (CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
results = requests.get(url).json()
venues  = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues    = nearby_venues.loc[:,filtered_columns]

# filter the category of the rows
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print( '{} venues were reported from Foursquare API call.'. format((nearby_venues.shape[0])))
nearby_venues.head()

16 venues were reported from Foursquare API call.


,name,categories,lat,lng
0,Rorschach Brewing Co.,Brewery,43.663483,-79.319824
1,Leslieville Farmers Market,Farmers Market,43.664901,-79.319784
2,The Sidekick,Comic Shop,43.664484,-79.325162
3,Queen Margherita Pizza,Pizza Place,43.664685,-79.324164
4,Chino Locos,Burrito Place,43.664653,-79.325584


In [15]:
# Question 3(contd.): create a function to obtain the venuew for all neighbourhoods in dataframe

def getNearbyVenues(names,latitudes,longitudes, radius = 500):
    venues_list = []
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
        # create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format (
                CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'], v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list ])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [16]:
# Question 3(contd.): Run the above code for each neigbourhood in tor_merged_df and create a new dataframe

tor_venues = getNearbyVenues(names = tor_merged_df['Neighbourhood'], latitudes = tor_merged_df['Latitude'], longitudes = tor_merged_df['Longitude'])

print(tor_venues.shape)
tor_venues.head()

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [17]:
# Question 3 (contd.): Examine how man venues returned for eachneighbourhood
tor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17


In [18]:
# Question 3(contd.): Let's examine the number of unique venues in the dataframe
print('There are {} unique venues from the toronto venues dataset.'. format (len(tor_venues['Venue Category'].unique())))

There are 210 unique venues from the toronto venues dataset.


In [19]:
# Question 3(contd.) : Analyze each neighbourhood in the dataframe
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix = " ", prefix_sep= " ")
# add column for neighbourhood and ensure it is the first column in the tor-onehot dataframe
tor_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 
fixed_columns = [tor_onehot.columns[-1]]+ list(tor_onehot.columns[: -1])
tor_onehot = tor_onehot[fixed_columns]

# the new dataframe has dimnensions
print (tor_onehot.shape)
# We further group rows of neighourhood by taking the mean of the frequency of the occurence of each category
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped

(1204, 211)


,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide, King, Richmond",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.000000,0.000000,0.060000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0000,0.000000,0.0000,0.02,0.040000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.060000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.040000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.0000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.020,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.02,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.040000,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.020000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.020000,0.02,0.000000

In [20]:
# Question 3(contd.): Let's display each neighbourhood along with the top 5 most common venues

num_top_venues = 10
for hood in tor_grouped['Neighbourhood']:
    print("----"+hood+ "------")
    temp = tor_grouped[tor_grouped['Neighbourhood']== hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp  = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond------
                   venue  freq
0       Asian Restaurant  0.06
1             Steakhouse  0.06
2                   Café  0.06
3                  Hotel  0.04
4            Pizza Place  0.04
5    American Restaurant  0.04
6            Coffee Shop  0.04
7              Gastropub  0.04
8         Breakfast Spot  0.04
9              Speakeasy  0.02


----Berczy Park------
                  venue  freq
0           Coffee Shop  0.08
1          Cocktail Bar  0.06
2              Beer Bar  0.04
3    Seafood Restaurant  0.04
4           Cheese Shop  0.04
5                  Café  0.04
6                Bakery  0.04
7        Farmers Market  0.04
8          Liquor Store  0.02
9    Belgian Restaurant  0.02


----Brockton, Exhibition Place, Parkdale Village------
                     venue  freq
0              Coffee Shop  0.10
1                     Café  0.10
2           Breakfast Spot  0.10
3            Grocery Store  0.05
4                   Bakery  0.05
5              

In [21]:
# Question 3(contd.): Putting the result above in a pandas dataframe
# The function below sorst the venues in descending order
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# Question 3(contd.): Cretae new dataframe that displays the top 10 venues for each neigbourhood
num_top_venues = 10
indicators = ['st','nd','rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{} {} Most Common Venue'. format(ind+1, indicators[ind]))
    except:
        columns.append('{} th Most Common Venue'. format(ind+1))
        
# New dataframe is created here
tor_venues_sorted = pd.DataFrame(columns=columns)
tor_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    tor_venues_sorted.iloc[ind,1:] = return_most_common_venues(tor_grouped.iloc[ind ,:], num_top_venues)
    
tor_venues_sorted.head()

,Neighbourhood,1 st Most Common Venue,2 nd Most Common Venue,3 rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
0,"Adelaide, King, Richmond",Asian Restaurant,Steakhouse,Café,American Restaurant,Gastropub,Breakfast Spot,Hotel,Coffee Shop,Pizza Place,Plaza
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Café,Cheese Shop,Beer Bar,Greek Restaurant,Concert Hall
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Climbing Gym,Burrito Place,Nightclub,Italian Restaurant,Intersection,Stadium,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Park,Pizza Place,Comic Shop,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry


In [23]:
# Question 3(contd.): Cluuster toronto neighbourhoods into  5 clusters using the kmeans 
kclusters = 3
tor_grouped_clustering = tor_grouped.drop('Neighbourhood', axis=1)
tor_grouped_clustering.head()
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(tor_grouped_clustering)
kmeans.labels_[0:10]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [24]:
# Question 3(contd.) : Now create a new datframe that includes the clusters and the top 10 venues for each neighbourhood
tor_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
# merge tor_grouped with tor_merged_df for each neighbourhood
tor_New_Merged = tor_merged_df
tor_New_Merged = tor_New_Merged.join(tor_venues_sorted.set_index('Neighbourhood'),on = 'Neighbourhood')
tor_New_Merged.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1 st Most Common Venue,2 nd Most Common Venue,3 rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Neighborhood,Pub,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Pub,Pizza Place,Lounge,Liquor Store,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Italian Restaurant,Intersection,Steakhouse,Fast Food Restaurant,Sushi Restaurant,Fish & Chips Shop,Brewery,Pub,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Music Store,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


In [25]:
# Question 3(contd.) : Visulaize resulting cluster
tor_map_clusters = folium.Map(location =[latitude, longitude], zoom_start = 11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [ i + x + (i*x)**2 for i in range (kclusters)]
colors_array = cm.rainbow(np.linspace (0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers
markers_colors = []
for lat,lon, poi, cluster in zip (tor_New_Merged['Latitude'], tor_New_Merged['Longitude'], tor_New_Merged['Neighbourhood'],tor_New_Merged['Cluster Labels'] ):
    label =folium.Popup(str(poi) +'Cluster'+ str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius = 5, popup = label, color = rainbow[cluster - 1], fill = True, fill_color = rainbow[cluster - 1 ], fill_opacity = 0.7 ). add_to(tor_map_clusters)
    
tor_map_clusters   

In [26]:
# Question 3 (contd.): Examine clusters
# cluster 1
tor_New_Merged.loc[tor_New_Merged['Cluster Labels'] == 0 , tor_New_Merged.columns[[1] + list (range(5, tor_New_Merged.shape[1]))]]

,Borough,Cluster Labels,1 st Most Common Venue,2 nd Most Common Venue,3 rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
4,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store
8,Central Toronto,0,Park,Gym,Playground,Trail,Restaurant,Creperie,Donut Shop,Dog Run,Discount Store,Diner
10,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store,Diner


In [27]:
# Question 3 (contd.): Examine clusters
# cluster 2
tor_New_Merged.loc[tor_New_Merged['Cluster Labels'] == 1 , tor_New_Merged.columns[[1] + list (range(5, tor_New_Merged.shape[1]))]]

,Borough,Cluster Labels,1 st Most Common Venue,2 nd Most Common Venue,3 rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
0,East Toronto,1,Trail,Health Food Store,Neighborhood,Pub,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Pub,Pizza Place,Lounge,Liquor Store,Juice Bar
2,East Toronto,1,Sandwich Place,Italian Restaurant,Intersection,Steakhouse,Fast Food Restaurant,Sushi Restaurant,Fish & Chips Shop,Brewery,Pub,Food & Drink Shop
3,East Toronto,1,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Music Store,Sandwich Place,Cheese Shop
5,Central Toronto,1,Park,Breakfast Spot,Gym,Convenience Store,Sandwich Place,Food & Drink Shop,Hotel,Department Store,Deli / Bodega,Dumpling Restaurant
6,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Bagel Shop,Gym / Fitness Center,Diner,Dessert Shop,Mexican Restaurant,Park,Chinese Restaurant
7,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Gym,Indoor Play Area,Discount Store
9,Central Toronto,1,Coffee Shop,Pub,Supermarket,Vietnamese Restaurant,Light Rail Station,Liquor Store,American Restaurant,Pizza Place,Sushi Restaurant,Fried Chicken Joint
11,Downtown Toronto,1,Coffee Shop,Restaurant,Market,Bakery,Pizza Place,Park,Italian Restaurant,Café,Pub,Playground
12,Downtown Toronto,1,Gastropub,Men's Store,Coffee Shop,Burger Joint,Gym,Restaurant,Gay Bar,Indian Restaurant,Ramen Restaurant,Bubble Tea Shop


In [28]:
# Question 3 (contd.): Examine clusters
# cluster 3
tor_New_Merged.loc[tor_New_Merged['Cluster Labels'] == 2 , tor_New_Merged.columns[[1] + list (range(5, tor_New_Merged.shape[1]))]]

,Borough,Cluster Labels,1 st Most Common Venue,2 nd Most Common Venue,3 rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
22,Central Toronto,2,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store,Diner
